# Table - Iteration time table

Note: This is a compiled SI figure of many sub-tables

In [1]:
import os
work_dir = "H:/workspace/ptyrad_paper"
os.chdir(work_dir)
print("Current working dir: ", os.getcwd())

Current working dir:  H:\workspace\ptyrad_paper


In [2]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import numpy as np
from itertools import product
from ptyrad.load import load_hdf5

In [3]:
def load_iter_times(package, batch, probe, slice):
    iter_times = []
    for round in range(1,6):
        base_path = '03_output/tBL_WSe2/'
        
        if package == 'ptyrad':
            package_path = f'20250613_{package}_table_r{round}/' # 0608 has positivity / sparsity; 0614 is constraint free just like Fig 3a
            ptyrad_postfix = '_Adam_plr1e-4_oalr5e-4_oplr5e-4_slr5e-4_orblur0.5_ozblur1_oathr0.98_sng1.0_aff1_0_-3_0'

            if slice == 1:
                slice_str = '1slice'
            else:
                slice_str = f'{int(slice)}slice_dz{12//int(slice)}'
                
            ptyrad_path = os.path.join(base_path, package_path, f'full_N16384_dp128_flipT100_random{batch}_p{probe}_1obj_{slice_str}{ptyrad_postfix}','model_iter0020.hdf5')
            try:
                iter_time = load_hdf5(ptyrad_path, key='avg_iter_t')
            except FileNotFoundError:
                iter_time = 'OOM'
            
        elif package == 'ptyshv':
            package_path = f'20250608_{package}_table_r{round}/'
            if slice ==1:
                slice_str = ''
            else:
                slice_str = f'_Ns{int(slice)}_dz{12//int(slice)}_reg1'
                
            ptyshv_path = os.path.join(base_path, package_path, 'roi1_Ndp128_step128', f'MLs_L1_p{probe}_g{batch}_pc1_noModel_mm{slice_str}_dpFlip_ud_T', 'Niter20.mat')
            try:
                iter_time = load_hdf5(ptyshv_path, key='outputs.avgTimePerIter').item()
            except FileNotFoundError:
                iter_time = 'OOM'
        
        elif package == 'py4dstem':
            package_path = f'20250608_{package}_table_r{round}/'
            
            if slice == 1:
                slice_str = '1slice'
                reg_str = ''
            else:
                slice_str = f'{int(slice)}slice_dz{12//int(slice)}'
                reg_str = '_kzf1'
                
            py4dstem_path = os.path.join(base_path, package_path, f'N16384_dp128_flipT100_random{batch}_p{probe}_{slice_str}_update0.5{reg_str}','model_iter0020.hdf5')
            try:
                iter_time = load_hdf5(py4dstem_path, key='iter_times').mean(0).item()
            except FileNotFoundError:
                iter_time = 'OOM'
        else:
            return ValueError(f"Incorrect {package}, please choose between 'ptyrad', 'ptyshv', or 'py4dstem'.")
        
        iter_times.append(iter_time)
    return iter_times

In [4]:
# Loading data
batch_sizes = [16, 32, 64, 128, 256, 512, 1024]
slices = [1, 3, 6]
probes = [1, 3, 6, 12]
packages = ['ptyrad', 'ptyshv', 'py4dstem']

data = []
for b, s, p, pkg in product(batch_sizes, slices, probes, packages):
    
    iter_times = load_iter_times(pkg, b, p, s)
    print(iter_times)

    if 'OOM' in iter_times:
        time_str = 'OOM'
    else:
        time_str = f"{np.mean(iter_times):.3g} ± {np.std(iter_times):.1g}"

    data.append({
        "Batch Size": b,
        "Slices": s,
        "Probes": p,
        "Package": pkg,
        "Time ± Std": time_str
    })

df = pd.DataFrame(data)

Success! Loaded .hdf5 file with key = 'avg_iter_t' from path = '03_output/tBL_WSe2/20250613_ptyrad_table_r1/full_N16384_dp128_flipT100_random16_p1_1obj_1slice_Adam_plr1e-4_oalr5e-4_oplr5e-4_slr5e-4_orblur0.5_ozblur1_oathr0.98_sng1.0_aff1_0_-3_0\model_iter0020.hdf5'
Success! Loaded .hdf5 file with key = 'avg_iter_t' from path = '03_output/tBL_WSe2/20250613_ptyrad_table_r2/full_N16384_dp128_flipT100_random16_p1_1obj_1slice_Adam_plr1e-4_oalr5e-4_oplr5e-4_slr5e-4_orblur0.5_ozblur1_oathr0.98_sng1.0_aff1_0_-3_0\model_iter0020.hdf5'
Success! Loaded .hdf5 file with key = 'avg_iter_t' from path = '03_output/tBL_WSe2/20250613_ptyrad_table_r3/full_N16384_dp128_flipT100_random16_p1_1obj_1slice_Adam_plr1e-4_oalr5e-4_oplr5e-4_slr5e-4_orblur0.5_ozblur1_oathr0.98_sng1.0_aff1_0_-3_0\model_iter0020.hdf5'
Success! Loaded .hdf5 file with key = 'avg_iter_t' from path = '03_output/tBL_WSe2/20250613_ptyrad_table_r4/full_N16384_dp128_flipT100_random16_p1_1obj_1slice_Adam_plr1e-4_oalr5e-4_oplr5e-4_slr5e-4_orbl

In [5]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

# Define color per package
package_colors = {
    "ptyrad": "#cfe2f3",     # light blue
    "ptyshv": "#fce5cd",     # light orange
    "py4dstem": "#d9ead3"   # light green
}
probe_header_color = "#E4E4E4"

def parse_val(val_str):
    try:
        if "OOM" in val_str:
            return float('inf')
        return float(val_str.split('±')[0].strip())
    except:
        return float('inf')

fig = plt.figure(figsize=(16.5, 10), dpi=600)
gs = gridspec.GridSpec(
    nrows=7+1,
    ncols=3+1,
    figure=fig,
    wspace=0,
    hspace=0,
    width_ratios=[0.1, 1, 1, 1], 
    height_ratios=[0.05, 1, 1, 1, 1, 1, 1, 1]
)

# Column headers
for j, s in enumerate(slices):
    ax = fig.add_subplot(gs[0, j+1])
    ax.axis("off")
    ax.text(0.42, 0, f"{s} slice{'s' if s > 1 else ''}",
            ha='center', va='center', fontsize=12, fontweight='bold')

# Row headers
for i, b in enumerate(batch_sizes):
    ax = fig.add_subplot(gs[i+1, 0])
    ax.axis("off")
    ax.text(0, 0.5, f"batch {b}",
            ha='center', va='center', fontsize=12, rotation=90, fontweight='bold')

# Main grid
for i, b in enumerate(batch_sizes):
    for j, s in enumerate(slices):
        ax = fig.add_subplot(gs[i+1, j+1])
        ax.axis("off")

        subset = df[(df["Batch Size"] == b) & (df["Slices"] == s)]
        pivot = subset.pivot(index="Package", columns="Probes", values="Time ± Std")

        formatted_probes = [f"{int(p)} probe{'s' if p > 1 else ''}" for p in pivot.columns]
        table_data = [formatted_probes] + pivot.values.tolist()
        row_labels = list(pivot.index)

        table = ax.table(
            cellText=table_data,
            rowLabels=[""] + row_labels,
            colLabels=None,
            cellLoc='center',
            loc='center'
        )
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(0.82, 1)

        # Color probe headers
        for col in range(len(formatted_probes)):
            header_cell = table[0, col]
            header_cell.set_facecolor(probe_header_color)
            # header_cell.set_text_props(fontweight='bold')

        # Color package label cells
        for row_idx, pkg in enumerate(row_labels):
            label_cell = table[row_idx + 1, -1]
            label_cell.set_facecolor(package_colors.get(pkg, '#eeeeee'))
            # label_cell.set_text_props(fontweight='bold')

        # Bold and color best values
        for col_idx, col_name in enumerate(pivot.columns):
            values = pivot[col_name].tolist()
            parsed = [parse_val(v) for v in values]
            if all(v == float('inf') for v in parsed):
                continue  # skip column if all OOM
            min_val = min(parsed)
            for row_idx, pkg in enumerate(row_labels):
                cell_val = str(pivot.iloc[row_idx, col_idx])
                cell = table[row_idx + 1, col_idx]
                if "OOM" in cell_val:
                    continue
                if parsed[row_idx] == min_val:
                    cell.set_text_props(weight='bold', color='black')
                    cell.set_facecolor(package_colors.get(pkg, '#dddddd'))

fig.suptitle("Iteration Time (sec) Benchmark", fontsize=20, y=1)
plt.tight_layout()
plt.savefig("05_figures/Fig_SXX_iter_time_table.pdf", bbox_inches="tight")
plt.savefig("05_figures/Fig_SXX_iter_time_table.png", bbox_inches="tight")

plt.show()
